In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

c:\Users\vedaa\anaconda3\envs\universal\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.19.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


Num GPUs Available:  0


In [2]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import time

# 1. Enable GPU growth to prevent memory issues
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 2. Verify GPU is available and configured
print("GPU Available:", tf.test.is_gpu_available())
print("GPU Device:", tf.test.gpu_device_name())

# 3. Create a simple model
def create_model(input_shape=(32, 32, 3)):
    model = models.Sequential([
        layers.Conv2D(8, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(16, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(10)
    ])
    return model

# 4. Create and compile model
model = create_model()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 5. Create tiny dataset (10 samples)
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_images = train_images[:10].astype('float32') / 255.0  # Only use 10 images
train_labels = train_labels[:10]
test_images = test_images[:10].astype('float32') / 255.0
test_labels = test_labels[:10]

# 6. Train with GPU
start_time = time.time()
model.fit(train_images, train_labels,
          epochs=5,
          batch_size=2,
          validation_data=(test_images, test_labels))
print(f"Training completed in {time.time() - start_time:.2f} seconds")

# 7. Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 8. Save the model
with open('gpu_trained_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model saved successfully")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available: False
GPU Device: 
   221184/170498071 [..............................] - ETA: 4:58:24

KeyboardInterrupt: 